In [45]:
import re
import wandb
import pandas as pd

username = "memmelma"
project = "vit-vo"

metric = "Objective/eval_all"
subtract_metric = "eval_all_unified_depth/aux_depth_loss"
subtract_metric = ""
step_threshold = 99 # 75 for tlr

name = "eval_vit_in21k_act"
name = name.replace('eval', 'vo')

name = "vo_vit_b_dino_act_rgbd"

In [46]:
# retreive runs

run_ids = []
run_names = []
run_ckpts = []

api = wandb.Api()
runs = api.runs(f"{username}/{project}")

for run in runs:
    if name in run.name:
        run_ids += [run.id]
        run_names += [run.name]
        run_ckpts += [run.config["config"]["CHECKPOINT_FOLDER"]]
run_ids, run_names

(['2dg03jwe', '1ccpf7yy'],
 ['vo_vit_b_dino_act_rgbd[usual_totem_31]',
  'vo_vit_b_dino_act_rgbd_freeze[eternal_tree_30]'])

In [47]:
# filter runs

# pattern = name + "\[.\/.]"
pattern = name + "\["

run_names_filter = []
run_ids_filter = []
run_ckpts_filter = []

for run_name, run_id, run_ckpt in zip(run_names,run_ids,run_ckpts):
    if re.match(pattern, run_name) is not None:
        run_names_filter += [run_name]
        run_ids_filter += [run_id]
        run_ckpts_filter += [run_ckpt]

run_names_filter

['vo_vit_b_dino_act_rgbd[usual_totem_31]']

In [48]:
# combine runs

infos = pd.DataFrame([])

for run_id, run_ckpt in zip(run_ids_filter, run_ckpts_filter):
    
    
    run = api.run(f"{username}/{project}/runs/{run_id}")
    
    history = run.history()
    select = history["_step"]<=step_threshold
    
#     info = pd.concat((history[metric][select], history["_step"][select]), axis=1)
    if subtract_metric != "":
        info = pd.concat((history[metric][select]-history[subtract_metric][select], history["_step"][select]), axis=1)
    else:
        info = pd.concat((history[metric][select], history["_step"][select]), axis=1)
        
    info.columns = [metric, "_step"]
    
    info["CHECKPOINT_FOLDER"] = run_ckpt

    infos = pd.concat((infos, info))

In [52]:
# get best run
best_run = infos[infos[metric] == infos[metric].min()]
print(name)
best_run

vo_vit_b_dino_act_rgbd


,Objective/eval_all,_step,CHECKPOINT_FOLDER
67,0.000679,67,train_log/vit/unique/vo_vit_b_dino_act_rgbd/ch...


In [8]:
# print checkpoint
print('\n      "regress_all_best": { \n        "all": "' + best_run["CHECKPOINT_FOLDER"].item() + f'/ckpt_epoch_{best_run["_step"].item()}'+'.pth",\n        },')


      "regress_all_best": { 
        "all": "train_log/vit/vo_vit_in21k_act_auxd_dt_20220408_120819651384_s_1/checkpoints/resume_20220412_103253291325/checkpoints/ckpt_epoch_133.pth",
        },
